# 소득세법 RAG 구축 with Gemini API
GEMINI_API_KEY를 활용하여 소득세법 관련 RAG 시스템을 구축합니다.

## 1. 환경 설정 및 라이브러리 설치

In [1]:
# 필요한 라이브러리 설치
!pip install google-generativeai langchain chromadb langchain-community langchain-google-genai

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB ? eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
     ---------------------------------------- 43.0/43.0 kB ? eta 0:00:00
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/76.3 k

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.45.1 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.


In [ ]:
# 라이브러리 임포트
import os
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 임포트 완료")

## 2. Gemini API 키 설정

In [ ]:
# Gemini API 키 설정 (환경변수 사용 권장)
api_key = os.environ.get('GEMINI_API_KEY')

if api_key:
    print(f"[OK] GEMINI_API_KEY 확인됨: {api_key[:10]}...")
    genai.configure(api_key=api_key)
else:
    print("[ERROR] GEMINI_API_KEY 환경변수가 설정되지 않았습니다.")
    print("설정 방법:")
    print("  Windows: set GEMINI_API_KEY=your_api_key_here")
    print("  Linux/Mac: export GEMINI_API_KEY=your_api_key_here")
    
    # 임시로 직접 입력하는 방법
    # api_key = input("GEMINI_API_KEY를 입력하세요: ")
    # genai.configure(api_key=api_key)

## 3. 소득세법 문서 데이터 준비

In [ ]:
# 소득세법 관련 문서 데이터 (예시)
# 실제 사용 시에는 공식 세법 문서나 해석 자료를 사용해야 합니다.

tax_documents = [
    {
        "title": "소득세법 제14조 (종합소득세의 과세표준)",
        "content": """
        종합소득세의 과세표준은 해당 과세기간의 종합소득금액에서 종합소득공제를 차감한 금액으로 한다.
        종합소득공제는 인적공제, 연금보험료공제, 특별소득공제, 그 밖의 소득공제 및 표준공제로 구분한다.
        """
    },
    {
        "title": "소득세법 제55조 (세율)", 
        "content": """
        종합소득에 대한 소득세는 해당 과세기간의 종합소득 과세표준에 다음 각 호의 세율을 적용하여 계산한 금액(이하 "종합소득산출세액"이라 한다)을 그 세액으로 한다.
        
        1. 1,400만원 이하: 과세표준의 6퍼센트
        2. 1,400만원 초과 5,000만원 이하: 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)
        3. 5,000만원 초과 8,800만원 이하: 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)
        4. 8,800만원 초과 1억5천만원 이하: 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)
        5. 1억5천만원 초과 3억원 이하: 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)
        6. 3억원 초과 5억원 이하: 9,406만원 + (3억원을 초과하는 금액의 40퍼센트)
        7. 5억원 초과 10억원 이하: 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)
        8. 10억원 초과: 3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)
        """
    },
    {
        "title": "근로소득공제",
        "content": """
        근로소득에서 다음 각 호의 금액을 공제한다.
        
        1. 500만원 이하: 총급여액의 70퍼센트
        2. 500만원 초과 1,500만원 이하: 350만원 + (500만원을 초과하는 금액의 40퍼센트)
        3. 1,500만원 초과 4,500만원 이하: 750만원 + (1,500만원을 초과하는 금액의 15퍼센트)
        4. 4,500만원 초과 1억원 이하: 1,200만원 + (4,500만원을 초과하는 금액의 5퍼센트)
        5. 1억원 초과: 1,475만원 + (1억원을 초과하는 금액의 2퍼센트)
        """
    },
    {
        "title": "인적공제",
        "content": """
        거주자(직전 과세기간에 종합소득금액이 100만원 이하인 거주자는 제외한다)에 대해서는 해당 과세기간에 다음 각 호의 금액을 종합소득금액에서 공제한다.
        
        1. 기본공제: 거주자 본인, 배우자, 생계를 같이하는 부양가족 1명당 연 150만원
        2. 추가공제: 경로우대자(70세 이상) 1명당 연 100만원, 장애인 1명당 연 200만원
        3. 부양가족 요건: 연간소득금액 100만원 이하, 연령 요건(직계존속 60세 이상, 직계비속 20세 이하)
        """
    },
    {
        "title": "연말정산 개념",
        "content": """
        연말정산은 근로자가 한 해 동안 납부한 소득세를 정산하는 절차입니다.
        매월 급여에서 원천징수한 세액과 실제 계산된 세액을 비교하여 차액을 환급하거나 추가 징수합니다.
        
        주요 공제 항목:
        - 인적공제 (본인, 배우자, 부양가족)
        - 연금보험료공제 (국민연금 등)
        - 특별소득공제 (건강보험료, 고용보험료 등)
        - 그 밖의 소득공제 (개인연금저축, 주택청약저축 등)
        - 세액공제 (근로소득세액공제, 자녀세액공제 등)
        """
    }
]

print(f"소득세법 문서 {len(tax_documents)}개 준비 완료")
for i, doc in enumerate(tax_documents):
    print(f"{i+1}. {doc['title']}")

## 4. 텍스트 분할 (Text Splitting)

In [ ]:
# 텍스트 분할기 설정 (교육자료에서 권장된 설정 적용)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    # 한 덩어리 최대 글자 수
    chunk_overlap=200,  # 앞뒤 청크 겹침으로 문맥 보존
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# 문서를 청크로 분할
from langchain.schema import Document

documents = []
for doc in tax_documents:
    # Document 객체 생성
    full_text = f"제목: {doc['title']}\n\n내용: {doc['content']}"
    document = Document(
        page_content=full_text,
        metadata={"title": doc['title'], "source": "소득세법"}
    )
    documents.append(document)

# 텍스트 분할 실행
splits = text_splitter.split_documents(documents)

print(f"원본 문서 {len(documents)}개 → 분할된 청크 {len(splits)}개")
print(f"첫 번째 청크 미리보기: {splits[0].page_content[:200]}...")

## 5. Gemini 임베딩 및 벡터 저장소 구축

In [ ]:
# Gemini 임베딩 모델 설정
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",  # 최신 Gemini 임베딩 모델
    google_api_key=api_key
)

print("Gemini 임베딩 모델 초기화 완료")

# 테스트 임베딩 생성
test_embedding = embeddings.embed_query("소득세 계산 테스트")
print(f"임베딩 차원: {len(test_embedding)}")
print(f"임베딩 샘플: {test_embedding[:3]}")

In [ ]:
# ChromaDB를 활용한 벡터 저장소 구축 (교육자료 권장 방식)
persist_directory = "./chroma_tax_db"  # 영구 저장 디렉토리

# 기존 데이터베이스 삭제 (새로 만들 경우)
import shutil
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
    print("기존 벡터 데이터베이스 삭제됨")

# 벡터 데이터베이스 생성
print("벡터 데이터베이스 생성 중... (시간이 소요될 수 있습니다)")

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    collection_name="tax_law_collection",
    persist_directory=persist_directory
)

print(f"[OK] 벡터 데이터베이스 생성 완료!")
print(f"[OK] 저장된 문서 수: {vectorstore._collection.count()}")
print(f"[OK] 저장 위치: {persist_directory}")

## 6. 리트리버 최적화 (MMR 방식 적용)

In [ ]:
# MMR(Maximal Marginal Relevance) 방식 리트리버 구성
# 교육자료에서 권장된 설정 적용
retriever = vectorstore.as_retriever(
    search_type="mmr",           # 다양성과 관련성 균형
    search_kwargs={
        "k": 8,                  # 최종 반환 문서 수
        "fetch_k": 24,           # 초기 검색 문서 수 
        "lambda_mult": 0.5       # 관련성 vs 다양성 균형 (0~1)
    }
)

print("MMR 리트리버 설정 완료")
print(f"검색 방식: MMR (Maximal Marginal Relevance)")
print(f"최종 반환: {retriever.search_kwargs['k']}개 문서")
print(f"초기 검색: {retriever.search_kwargs['fetch_k']}개 문서")
print(f"다양성 가중치: {retriever.search_kwargs['lambda_mult']}")

In [ ]:
# 리트리버 테스트
test_query = "5천만원 소득이 있을 때 소득세를 계산해주세요"
retrieved_docs = retriever.get_relevant_documents(test_query)

print(f"테스트 쿼리: {test_query}")
print(f"검색된 문서 수: {len(retrieved_docs)}")
print("\n=== 검색 결과 미리보기 ===")
for i, doc in enumerate(retrieved_docs[:3]):
    print(f"{i+1}. {doc.metadata.get('title', '제목없음')}")
    print(f"   {doc.page_content[:100]}...\n")

## 7. Gemini Chat 모델 설정

In [ ]:
# Gemini Chat 모델 설정
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",      # 최신 Gemini 모델
    google_api_key=api_key,
    temperature=0.1,             # 일관된 답변을 위해 낮은 temperature
    convert_system_message_to_human=True
)

print("Gemini Chat 모델 설정 완료")

# 모델 테스트
test_response = llm.invoke("안녕하세요! 소득세 계산 도우미입니다.")
print(f"모델 응답 테스트: {test_response.content[:100]}...")

## 8. RAG 프롬프트 설계 (교육자료 기반)

In [ ]:
# 소득세법 전용 프롬프트 템플릿 설계
# 교육자료의 쿼리 최적화 방법론 적용

tax_prompt_template = """
당신은 대한민국 소득세법 전문가입니다. 
주어진 소득세법 관련 자료를 바탕으로 정확하고 상세한 답변을 제공해주세요.

답변 시 다음 사항을 준수해주세요:
1. 구체적인 세율과 계산 공식을 명시하세요
2. 단계별 계산 과정을 보여주세요  
3. 관련 세법 조항을 인용하세요
4. 예시 금액으로 실제 계산을 보여주세요
5. 정보가 불충분한 경우 그 사실을 명시하세요

참고 자료:
{context}

질문: {question}

답변:
"""

# 프롬프트 템플릿 생성
tax_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=tax_prompt_template
)

print("소득세법 전용 프롬프트 템플릿 생성 완료")

## 9. 쿼리 정규화 함수 (교육자료 방법론 적용)

In [ ]:
# 교육자료에서 제시된 쿼리 정규화 기법 구현
def normalize_tax_query(query: str) -> str:
    """
    소득세 질문을 검색에 최적화된 형태로 변환
    교육자료의 핵심 인사이트 적용
    """
    # 숫자 표기 통일
    query = query.replace("천만원", ",000만원")
    query = query.replace("억원", ",0000만원")
    
    # 용어 정규화
    query = query.replace("연봉", "과세표준")
    query = query.replace("월급", "근로소득")
    query = query.replace("세금", "소득세")
    
    # 계산 컨텍스트 강화 (교육자료 핵심 인사이트!)
    calculation_keywords = ["계산", "얼마", "세율", "공제", "누진"]
    if not any(keyword in query for keyword in calculation_keywords):
        query = query + " 계산 기준과 누진공제를 적용해 계산"
    
    # "얼마인가요?" → "계산하면 얼마인가요?" (교육자료 핵심 개선사항)
    if "얼마인가요" in query and "계산" not in query:
        query = query.replace("얼마인가요", "계산하면 얼마인가요")
    
    return query.strip()

# 테스트
test_queries = [
    "5천만원 소득이 있을 때 세금은 얼마인가요?",
    "연봉 8천만원인 사람의 소득세는?",
    "1억원 과세표준의 세율은?"
]

print("=== 쿼리 정규화 테스트 ===")
for query in test_queries:
    normalized = normalize_tax_query(query)
    print(f"원본: {query}")
    print(f"정규화: {normalized}\n")

## 10. RAG 체인 구축 (LCEL 파이프라인)

In [ ]:
# RAG 체인 구축 (LCEL 파이프라인 방식)
def format_docs(docs):
    """문서들을 하나의 컨텍스트로 포맷팅"""
    return "\n\n".join([f"문서 {i+1}: {doc.page_content}" for i, doc in enumerate(docs)])

# LCEL 체인 구성
rag_chain = (
    {
        "context": retriever | format_docs,  # 검색 + 포맷팅
        "question": RunnablePassthrough()    # 질문 그대로 전달
    }
    | tax_prompt          # 프롬프트 적용
    | llm                 # LLM 호출
    | StrOutputParser()   # 문자열 파싱
)

print("RAG 체인 구축 완료")
print("파이프라인: 질문 입력 → 쿼리 정규화 → 벡터 검색 → 프롬프트 생성 → Gemini 응답")

## 11. 소득세법 RAG 시스템 테스트

In [ ]:
# RAG 시스템 기본 기능 테스트
def ask_tax_question(question: str, show_context: bool = False):
    """
    소득세 질문에 대한 RAG 답변 생성
    """
    print(f"=== 질문: {question} ===")
    
    # 1. 쿼리 정규화 적용
    normalized_query = normalize_tax_query(question)
    if normalized_query != question:
        print(f"[정규화] {normalized_query}")
    
    # 2. 관련 문서 검색 (참고용)
    if show_context:
        docs = retriever.get_relevant_documents(normalized_query)
        print(f"\n[검색된 문서 {len(docs)}개]")
        for i, doc in enumerate(docs[:2]):
            print(f"{i+1}. {doc.metadata.get('title', '제목없음')}")
    
    # 3. RAG 체인으로 답변 생성
    try:
        answer = rag_chain.invoke(normalized_query)
        print(f"\n[답변]\n{answer}")
    except Exception as e:
        print(f"\n[오류] {e}")
    
    print("\n" + "="*80 + "\n")

# 테스트 질문들
test_questions = [
    "5천만원 소득이 있을 때 소득세를 계산해주세요",
    "근로소득공제는 어떻게 계산하나요?",
    "인적공제에는 어떤 것들이 있나요?"
]

print("=== 소득세법 RAG 시스템 테스트 시작 ===\n")

for question in test_questions:
    ask_tax_question(question, show_context=True)
    
print("=== RAG 시스템 테스트 완료 ===")

## 12. 인터랙티브 세법 상담 시스템

In [ ]:
# 대화형 소득세 상담 시스템
def tax_consultation_system():
    """
    대화형 소득세 상담 시스템
    """
    print("🏛️ 소득세법 AI 상담사에 오신 것을 환영합니다!")
    print("💡 소득세 계산, 공제, 세율 등 궁금한 것을 물어보세요.")
    print("⚠️  실제 세무 신고 시에는 반드시 전문가와 상담하시기 바랍니다.")
    print("🔚 종료하려면 'quit' 또는 '종료'를 입력하세요.\n")
    
    while True:
        try:
            # 사용자 질문 입력
            user_question = input("💬 질문을 입력하세요: ").strip()
            
            # 종료 조건
            if user_question.lower() in ['quit', 'exit', '종료', '그만', 'q']:
                print("\n👋 상담을 종료합니다. 감사합니다!")
                break
            
            # 빈 질문 체크
            if not user_question:
                print("❌ 질문을 입력해주세요.\n")
                continue
            
            # RAG 답변 생성
            print("\n🤖 답변 생성 중...")
            normalized_query = normalize_tax_query(user_question)
            answer = rag_chain.invoke(normalized_query)
            
            print(f"\n📋 답변:\n{answer}\n")
            print("-" * 60)
            
        except KeyboardInterrupt:
            print("\n\n👋 상담을 종료합니다. 감사합니다!")
            break
        except Exception as e:
            print(f"\n❌ 오류가 발생했습니다: {e}\n")
            continue

# 상담 시스템 시작 (주석 해제하여 실행)
# tax_consultation_system()

print("💡 위의 tax_consultation_system() 함수 주석을 해제하면 대화형 상담을 시작할 수 있습니다.")

## 13. 성능 평가 및 개선사항

In [ ]:
# RAG 시스템 성능 평가
def evaluate_rag_performance():
    """
    RAG 시스템 성능 평가 (교육자료 방법론 기반)
    """
    print("=== RAG 시스템 성능 평가 ===")
    
    # 1. 벡터 데이터베이스 통계
    doc_count = vectorstore._collection.count()
    print(f"📊 저장된 문서 수: {doc_count}")
    
    # 2. 임베딩 정보
    print(f"🔢 임베딩 모델: text-embedding-004 (Gemini)")
    print(f"📏 임베딩 차원: {len(test_embedding)}")
    
    # 3. 검색 성능 테스트
    search_test_queries = [
        "소득세 세율",
        "근로소득공제 계산", 
        "인적공제 금액",
        "연말정산 방법"
    ]
    
    print(f"\n🔍 검색 테스트 결과:")
    for query in search_test_queries:
        docs = retriever.get_relevant_documents(query)
        print(f"  '{query}' → {len(docs)}개 문서 검색됨")
    
    # 4. 응답 시간 측정
    import time
    
    start_time = time.time()
    test_answer = rag_chain.invoke("5천만원 소득세 계산")
    response_time = time.time() - start_time
    
    print(f"\n⏱️  평균 응답 시간: {response_time:.2f}초")
    print(f"📝 응답 길이: {len(test_answer)}자")
    
    # 5. 개선 권장사항
    print(f"\n💡 개선 권장사항:")
    print(f"  - 더 많은 소득세법 문서 추가")
    print(f"  - 실제 세무사례 데이터 확충")
    print(f"  - LangSmith를 활용한 정량적 평가")
    print(f"  - 사용자 피드백 수집 시스템 구축")

# 성능 평가 실행
evaluate_rag_performance()

## 14. 실습 완료 및 다음 단계

In [ ]:
print("🎉 소득세법 RAG 구축 실습 완료! 🎉")
print()
print("✅ 완료된 작업:")
print("  1. Gemini API 키 설정 및 인증")
print("  2. 소득세법 문서 데이터 준비")
print("  3. 텍스트 분할 (chunk_size=1500, overlap=200)")
print("  4. Gemini 임베딩 모델 활용 벡터화")
print("  5. ChromaDB 기반 벡터 저장소 구축")
print("  6. MMR 방식 리트리버 최적화")
print("  7. 쿼리 정규화 함수 구현 (교육자료 방법론)")
print("  8. LCEL 파이프라인 기반 RAG 체인 구축")
print("  9. 대화형 상담 시스템 개발")
print(" 10. 성능 평가 및 개선방안 제시")
print()
print("🔧 사용된 주요 기술:")
print(f"  - 임베딩: Gemini text-embedding-004 ({len(test_embedding)}차원)")
print("  - LLM: Gemini-1.5-pro")
print("  - 벡터DB: ChromaDB (로컬 영구 저장)")
print("  - 검색: MMR (관련성+다양성 균형)")
print("  - 프레임워크: LangChain + LCEL")
print()
print("📈 다음 단계 권장사항:")
print("  1. 더 많은 소득세법 조문 데이터 확충")
print("  2. 실제 세무 사례 및 해석 자료 추가")
print("  3. LangSmith를 활용한 체계적 성능 평가")
print("  4. 웹 인터페이스 개발 (Streamlit/FastAPI)")
print("  5. 실시간 세법 업데이트 반영 시스템")
print()
print("⚠️  주의사항:")
print("  - 이는 학습 목적의 데모 시스템입니다")
print("  - 실제 세무 신고 시에는 반드시 전문가와 상담하세요")
print("  - 세법은 수시로 개정되므로 최신 정보 확인이 필요합니다")
print()
print("🎯 석이, RAG 시스템 구축 실습을 성공적으로 완료하셨습니다!")